In [9]:
import time
import json
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from bs4 import BeautifulSoup

def lotte_eatz_crawler_stable():
    driver = webdriver.Chrome()
    main_url = "https://www.lotteeatz.com/brand/ria"
    driver.get(main_url)
    driver.maximize_window()

    all_burger_data = []

    try:
        # 쿠키 배너 처리 (안정성을 위해 유지)
        try:
            cookie_agree_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), '동의')]"))
            )
            cookie_agree_button.click()
            print("✅ 쿠키 동의 배너를 클릭했습니다.")
        except TimeoutException:
            print("ℹ️ 쿠키 동의 배너가 없거나 이미 처리되었습니다.")

        # '버거' 카테고리 탭 요소 찾기
        burger_tab = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//button[contains(text(), '버거')]"))
        )

        ### (결정적 수정) 자바스크립트를 사용하여 강제로 클릭 ###
        # 분기점: [버거 탭 요소 확인] -> [자바스크립트로 클릭 이벤트 직접 실행]
        driver.execute_script("arguments[0].click();", burger_tab)
        print("✅ '버거' 카테고리 탭을 자바스크립트로 클릭했습니다.")

        # 상품 목록이 로드될 때까지 명시적으로 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'prod-list'))
        )
        time.sleep(1) # JS 클릭 후 DOM이 안정화될 시간을 추가로 줌

        # 반복 크롤링 (이하 로직은 이전과 거의 동일)
        product_count = len(driver.find_elements(By.CSS_SELECTOR, 'ul.prod-list > li.prod-item'))
        if product_count == 0:
            print("⚠️ 버거 탭 클릭 후 상품 목록을 찾을 수 없습니다. 사이트 구조를 확인해주세요.")
            return None

        print(f"총 {product_count}개의 버거 상품을 찾았습니다. 크롤링을 시작합니다.")

        for i in range(product_count):
            try:
                products = WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'ul.prod-list > li.prod-item'))
                )

                target_product = products[i]
                # 클릭 안정성을 위해 스크롤과 JS 클릭을 모두 사용
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", target_product)
                time.sleep(0.5)

                temp_soup = BeautifulSoup(target_product.get_attribute('innerHTML'), 'html.parser')
                temp_name = temp_soup.select_one('p.prod-name').text.strip()

                # 상세 페이지 이동도 안정성을 위해 JS 클릭 사용 가능
                driver.execute_script("arguments[0].click();", target_product)
                print(f"({i+1}/{product_count}) ➡️ '{temp_name}' 상세 페이지로 이동...")

                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "prod-tit")))

                # (정보 추출 로직은 동일)
                # ... (이전 코드와 동일한 추출 로직) ...

                driver.back()
                print(f"({i+1}/{product_count}) ⬅️ 메인 페이지로 복귀.")

                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'prod-list')))

            except (StaleElementReferenceException, TimeoutException) as e:
                print(f"({i+1}/{product_count}) ⚠️ 루프 중 오류 발생: {type(e).__name__}. 다음 항목으로 건너뜁니다.")
                # 문제가 발생하면 메인 페이지로 돌아가서 다음 루프를 시도
                driver.get(main_url)
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), '버거')]"))).click()
                time.sleep(1)
                continue

    except Exception as e:
        print(f"크롤링 중 심각한 오류가 발생했습니다: {e}")
    finally:
        driver.quit()
        print("\n브라우저를 종료했습니다.")
        return all_burger_data

# (데이터 저장 함수 save_data는 이전 코드와 동일)
# --- 3단계: 데이터 저장 ---
def save_data(data):
    """수집된 데이터를 JSON과 CSV 파일로 저장합니다."""
    # JSON 저장
    with open("lotte_eatz_burgers.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print("✅ JSON 파일 저장 완료: lotte_eatz_burgers.json")

    # CSV 저장을 위해 데이터 평탄화
    csv_data = []
    for item in data:
        # 영양정보와 원산지 정보를 보기 좋게 문자열로 변환
        nutrition_str = ", ".join([f"{k}: {v}" for k, v in item['nutrition'].items()])
        origin_str = ", ".join([f"{k}: {v}" for k, v in item['origin'].items()])

        csv_data.append([
            item['id'],
            item['name'],
            nutrition_str,
            item['allergy'],
            origin_str
        ])

    # CSV 저장
    with open("lotte_eatz_burgers.csv", "w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(['ID', '이름', '영양정보', '알러지정보', '원산지정보'])
        writer.writerows(csv_data)
    print("✅ CSV 파일 저장 완료: lotte_eatz_burgers.csv")


# --- 메인 실행 블록 ---
if __name__ == '__main__':
    burger_details = lotte_eatz_crawler_stable()

    if burger_details:
        save_data(burger_details)
        print("\n🎉 모든 작업이 성공적으로 완료되었습니다!")
    else:
        print("\n수집된 데이터가 없습니다.")

ℹ️ 쿠키 동의 배너가 없거나 이미 처리되었습니다.
크롤링 중 심각한 오류가 발생했습니다: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff622e4fca5+79861]
	GetHandleVerifier [0x0x7ff622e4fd00+79952]
	(No symbol) [0x0x7ff622bccada]
	(No symbol) [0x0x7ff622c24457]
	(No symbol) [0x0x7ff622c2471c]
	(No symbol) [0x0x7ff622c78217]
	(No symbol) [0x0x7ff622c4cb1f]
	(No symbol) [0x0x7ff622c74f8b]
	(No symbol) [0x0x7ff622c4c8b3]
	(No symbol) [0x0x7ff622c15272]
	(No symbol) [0x0x7ff622c16043]
	GetHandleVerifier [0x0x7ff62310b9dd+2946349]
	GetHandleVerifier [0x0x7ff623105c5a+2922410]
	GetHandleVerifier [0x0x7ff6231259e7+3052855]
	GetHandleVerifier [0x0x7ff622e6aa8e+189918]
	GetHandleVerifier [0x0x7ff622e72a2f+222591]
	GetHandleVerifier [0x0x7ff622e58ac4+116244]
	GetHandleVerifier [0x0x7ff622e58c79+116681]
	GetHandleVerifier [0x0x7ff622e3f058+11176]
	BaseThreadInitThunk [0x0x7ffc307c7374+20]
	RtlUserThreadStart [0x0x7ffc31d7cc91+33]


브라우저를 종료했습니다.

수집된 데이터가 없습니다.
